In [1]:
import pandas as pd
import numpy as np
from scipy.stats import bernoulli
from scipy.stats import linregress
import math

## Missing Data Management and using Pandas

### Step 1: Understanding how we can end up with NaNs
As a first step, let us initiate a dataframe with two columns $X_1$ and $X_2$.

In [2]:
df = pd.DataFrame(columns=['X1', 'X2'], dtype=float)
df

,X1,X2


The dataframe is initialized, however, there is no row there. Let us create a new row whose $X_1$ value is equal to $1$.

In [3]:
new_row = {'X1':1}

# append this row to the dataframe
# Note: append method in the latest versions of pandas has been made private (df._append) 
# df = df.append(new_row, ignore_index=True)


new_row_df = pd.DataFrame(new_row, index=[0])

df = pd.concat([df, new_row_df], ignore_index=True)

In [4]:
df

,X1,X2
0,1.0,NaN


As we see, the $X_2$ value is NaN (Not a Number) which is traditionally used to model missing data. This approach is useful as otherwise we would not be able to add a row (i.e., if we require all the information to be complete, then the row above will not be appended).

We can similarly assign a name to a new row. For example, let's add the same row again and name it `Alice`

In [5]:
df.loc["Alice"] = new_row
df

,X1,X2
0,1.0,NaN
Alice,1.0,NaN


We can change the $X_2$ value of Alice if it is not NaN. For example:

In [6]:
df.loc["Alice"]['X2'] = 5
df

,X1,X2
0,1.0,NaN
Alice,1.0,5.0


If we want to change the name of the first row, we can apply:

In [7]:
df.rename(index={0:'Bob'},inplace=True)
df

,X1,X2
Bob,1.0,NaN
Alice,1.0,5.0


Let us add one more row, this time with missing $X_1$. Note that the Numpy comment `np.nan` creates NaN automatically:

In [8]:
df.loc['Charlie'] = [np.nan, -1]
df

,X1,X2
Bob,1.0,NaN
Alice,1.0,5.0
Charlie,NaN,-1.0


### Step 2: Operations on NaNs

In [9]:
na_one, na_two = df.loc["Bob"] #Take Bob rows
print("We have", na_one, "and", na_two, "values")

We have 1.0 and nan values


In [10]:
print("Sum of", na_one, "and", na_two, "gives", na_one + na_two)

Sum of 1.0 and nan gives nan


In [11]:
print("Multiplication of", na_one, "and", na_two, "gives", na_one * na_two)

Multiplication of 1.0 and nan gives nan


In [12]:
print("Comparison of", na_one, "and", na_two, "gives", na_one == na_two)

Comparison of 1.0 and nan gives False


In [13]:
print("Comparison of", na_two, "and", na_two, "gives", na_two == na_two, "that is, NaN is not even equal to itself")

Comparison of nan and nan gives False that is, NaN is not even equal to itself


We usually ask if a value is nan as the following:

In [14]:
np.isnan(na_two) #use numpy

True

In [15]:
math.isnan(na_two) #use math

True

In [16]:
pd.isnull(na_two) #use pandas 

True

Moreover, oficially NaN is stored as a floating number

In [17]:
type(na_two)

float

So if we compare the types of NaN and a number, you get

In [18]:
type(na_one) == type(na_two)

True

### Step 3: Using Pandas to work with NaNs

The following drops all the rows of `df` where at least one of the values is missing, and creates a new dataframe

In [19]:
df2 = df.dropna()
df2

,X1,X2
Alice,1.0,5.0


Or, we can drop all the rows where $X_1$ value is NaN

In [20]:
df2 = df.dropna(subset = ["X1"])
df2

,X1,X2
Bob,1.0,NaN
Alice,1.0,5.0


In [21]:
df2 = df.dropna(subset = ["X2"]) #similar for X2
df2

,X1,X2
Alice,1.0,5.0
Charlie,NaN,-1.0


Add a new column based on $X_2$

In [22]:
df["X3"]  = df["X2"]*2
df

,X1,X2,X3
Bob,1.0,NaN,NaN
Alice,1.0,5.0,10.0
Charlie,NaN,-1.0,-2.0


Or, alternatively

In [23]:
df["X3"] = df.apply(lambda row: row["X2"]*2, axis=1)
df

,X1,X2,X3
Bob,1.0,NaN,NaN
Alice,1.0,5.0,10.0
Charlie,NaN,-1.0,-2.0


If we want special treatment to NaN values, for example, if we want to initalize $X_3$ value as $0$ if $X_2$ is NaN, we can apply:

In [24]:
df["X3"] = df.apply(lambda row: row["X2"]*2 if ~np.isnan(row["X2"]) else 0, axis=1)
df

,X1,X2,X3
Bob,1.0,NaN,0.0
Alice,1.0,5.0,10.0
Charlie,NaN,-1.0,-2.0


### Step 4: Exercise from the course
Consider a database where each record has $100$ fields. Assume further that for each record, each of the $100$ fields has a $1$ per cent chance of being empty, i.e. its value is missing.

If we were to remove all records with one or more empty fields, how many records would we remove?

In [25]:
#Step i) Generate a matrix of rowx * 100 columns
nr_rows = 10000 #let's say we have this many rows
data_to_use = np.ones(nr_rows*100) #generate all-ones; numpy.ones(shape, dtype=None, order='C', *, like=None)[source] Return a new array of given shape and type, filled with ones.

df_exercise = pd.DataFrame(data_to_use.reshape(nr_rows, 100)) #reshape the data so that we have rows * 100
df_exercise

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9996,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9997,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9998,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [26]:
#Step ii) Generate indices of missing values
missing_or_not = bernoulli.rvs((1/100), size=nr_rows*100) #keeps whether or not each element of df_exercise is missing
missing_or_not = missing_or_not.reshape(nr_rows, 100) #reshape so that index of missing values correspond to df_exercise

In [27]:
missing_rows, missing_cols = np.where(missing_or_not == 1) #indices of missing values

In [28]:
df_exercise.values[missing_rows, missing_cols] = np.nan #make the values missing for those indices

In [58]:
df_exercise

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9996,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9997,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9998,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [30]:
print("In total, there are" ,np.sum(df_exercise.isnull().sum()),"missing values")
#It should match `np.size(missing_rows)`

In total, there are 10024 missing values


In [59]:

#Let's see how many rows have at least one NaN
df_missing = df_exercise.isnull().any(axis=1) #each tow has True if at least one element is nan, False otherwise

In [64]:
df_exercise.isnull().any(axis=1)

0        True
1        True
2        True
3       False
4        True
        ...  
9995     True
9996     True
9997     True
9998    False
9999     True
Length: 10000, dtype: bool

In [32]:
print("There are", np.sum(df_missing), "rows with at least one missing data")
print("So if we drop rows with missing values, we lose", np.sum(df_missing)/nr_rows, "fraction of the whole data")

There are 6347 rows with at least one missing data
So if we drop rows with missing values, we lose 0.6347 fraction of the whole data


In [33]:
print("In other words, the following has", nr_rows - np.sum(df_missing) ,"rows")
df_exercise.dropna()

In other words, the following has 3653 rows


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
11,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
14,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9984,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9985,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9986,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9993,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


Of course, what we have just observed can be derived mathematically. Namely, we are interested in the probability of a single row to have no NaNs. Let $E$ be the event where all the $100$ variables are not-NaN for an arbitrary row. We then have
\begin{align*}
\mathbb{P}[E] = \prod_{i=1}^{100} \mathbb{P}[\text{i-th variable is not NaN}] & = \prod_{i=1}^{100} (1 - 1/100) \\
& = \left(\dfrac{99}{100}\right)^{100} = (0.99)^{100}.
\end{align*}
Notice that the first equality holds as each variable has a 1 per cent probability of being NaN *independently*.

In [34]:
prob_E = 0.99 ** 100
np.round(prob_E,3) #round to three decimals

0.366

Since we have `nr_rows` rows in the experiment above, we except to have $0.99 *$ `nr_rows` many rows after we drop the rows with at least one NaN values. Let's see if this is correct.

In [35]:
print("We expect to have", round(nr_rows * prob_E), "rows that are not dropped, and our experiment ended with", nr_rows - np.sum(df_missing), "rows")

We expect to have 3660 rows that are not dropped, and our experiment ended with 3653 rows


Although our expectation and the result of experiment are close to each other, they are not the same number. This is due to random sampling. We will see in the upcoming modules that if we increase the number of rows in this experiment, then the number of rows that are not dropped in the experiment will be equal to $(0.99)^{100}$ as we derived.

### Step 5: Applications on real-life data

Let us work on the following problem.

Dataset: https://www.statlearning.com/s/College.csv
Variables are explained at [this link](https://lse-me314.github.io/solutions/ME314_assignment1_solution.html).

We modify the dataset and provide you with a version which has missing values. It is named as "college_missing.csv"

# Questions:
### Missing values
#### a) Read the dataset by using Pandas and have a first look
#### b) Find out which values are missing
#### c) Replace the missing values with the mean for the feature
#### d) Figure out what is wrong with doing (c)
#### e) Replace the missing values using a linear model to estimate the values

# Solutions:
#### a) 

In [36]:
# df_college = pd.read_csv("college_missing.csv",index_col=0) #read the data
df_college = pd.read_csv("./W2/PCMLAI-M2-college_missing.csv",index_col=0) #read the data

In [37]:
df_college.head() #inspect

,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,PhD,S.F.Ratio,Grad.Rate
Abilene Christian University,1660.0,1232.0,721,23,52,2885,537,70,18.1,60
Adelphi University,2186.0,1924.0,512,16,29,2683,1227,29,12.2,56
Adrian College,1428.0,1097.0,336,22,50,1036,99,53,12.9,54
Agnes Scott College,417.0,349.0,137,60,89,510,63,92,7.7,59
Alaska Pacific University,NaN,146.0,55,16,44,249,869,76,11.9,15


#### b)

In [38]:
df_college.isna().sum() #how many missing values under each column

Apps           6
Accept         7
Enroll         0
Top10perc      0
Top25perc      0
F.Undergrad    0
P.Undergrad    0
PhD            0
S.F.Ratio      0
Grad.Rate      0
dtype: int64

In [39]:
missing_apps = df_college[df_college['Apps'].isnull()].index.tolist() #missing applications list (indexes)
missing_accepts = df_college[df_college['Accept'].isnull()].index.tolist() #missing accepts list (indexes)

In [40]:
missing_apps #names

['Alaska Pacific University',
 'Columbia College MO',
 'James Madison University',
 'Pacific Lutheran University',
 'University of the South',
 'Virginia Tech']

In [41]:
missing_accepts #names

['Chatham College',
 'Mount Mary College',
 'Muskingum College',
 'Pennsylvania State Univ. Main Campus',
 'University of Delaware',
 'University of Georgia',
 'University of North Carolina at Asheville']

#### c)

In [42]:
mean_of_apps = df_college["Apps"].mean() #compute mean of apps -- pandas automatically excludes NaNs!
mean_of_accepts = df_college["Accept"].mean() #compute mean of accepts -- pandas automatically excludes NaNs!
df_college_v1 = df_college.copy()#copy the dataframe as we will fill the missing data (we still want to keep original, so we copy it to a new one) 
df_college_v1.loc[df_college_v1["Apps"].isnull(), 'Apps']  = mean_of_apps #whenever there is missing data on "apps" change it with the mean
df_college_v1.loc[df_college_v1["Accept"].isnull(), 'Accept']  = mean_of_accepts #same for accepts 

In [43]:
df_college_v1

,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,PhD,S.F.Ratio,Grad.Rate
Abilene Christian University,1660.000000,1232.0,721,23,52,2885,537,70,18.1,60
Adelphi University,2186.000000,1924.0,512,16,29,2683,1227,29,12.2,56
Adrian College,1428.000000,1097.0,336,22,50,1036,99,53,12.9,54
Agnes Scott College,417.000000,349.0,137,60,89,510,63,92,7.7,59
Alaska Pacific University,2985.451362,146.0,55,16,44,249,869,76,11.9,15
...,...,...,...,...,...,...,...,...,...,...
Worcester State College,2197.000000,1515.0,543,4,26,3089,2029,60,21.0,40
Xavier University,1959.000000,1805.0,695,24,47,2849,1107,73,13.3,83
Xavier University of Louisiana,2097.000000,1915.0,695,34,61,2793,166,67,14.4,49
Yale University,10705.000000,2453.0,1317,95,99,5217,83,96,5.8,99


In [44]:
df_college_v1.isna().sum() #all fixed

Apps           0
Accept         0
Enroll         0
Top10perc      0
Top25perc      0
F.Undergrad    0
P.Undergrad    0
PhD            0
S.F.Ratio      0
Grad.Rate      0
dtype: int64

#### d)

In [45]:
df_college_v1.loc[missing_apps, ["Apps", "Accept"]] #we accepted more than the applications in some unis! This is wrong

,Apps,Accept
Alaska Pacific University,2985.451362,146.0
Columbia College MO,2985.451362,158.0
James Madison University,2985.451362,5285.0
Pacific Lutheran University,2985.451362,1392.0
University of the South,2985.451362,966.0
Virginia Tech,2985.451362,11719.0


In [46]:
df_college_v1.loc[missing_accepts, ["Apps", "Accept"]] #we accepted more than the applications in some unis! This wrong

,Apps,Accept
Chatham College,212.0,1996.837662
Mount Mary College,235.0,1996.837662
Muskingum College,1109.0,1996.837662
Pennsylvania State Univ. Main Campus,19315.0,1996.837662
University of Delaware,14446.0,1996.837662
University of Georgia,11220.0,1996.837662
University of North Carolina at Asheville,1757.0,1996.837662


#### e)

In [47]:
df_college_v2 = df_college.copy() #copy to a new one (we will use lin-reg)

In [48]:
df_college_v2_complete = df_college_v2.dropna() #drop nans as we want to fit a linear model 
df_college_v2_complete = df_college_v2_complete[["Apps", "Accept"]] #take only relevant columns

In [49]:
df_college_v2_complete.head() #have a look

,Apps,Accept
Abilene Christian University,1660.0,1232.0
Adelphi University,2186.0,1924.0
Adrian College,1428.0,1097.0
Agnes Scott College,417.0,349.0
Albertson College,587.0,479.0


In [50]:
slope_predict_accept, intercept_predict_accept, r_value, p_value, std_err = \
    linregress(df_college_v2_complete["Apps"],df_college_v2_complete["Accept"]) #use scipy to fit a linear regression
#this model explains the "accepts" by using "Apps" as predictors

In [51]:
slope_predict_apps, intercept_predict_apps, r_value, p_value, std_err = \
    linregress(df_college_v2_complete["Accept"],df_college_v2_complete["Apps"]) #use scipy
#this model explains the "Apps" by using "accepts" as predictors

In [52]:
#Let's fill missing values now based on the linear model
df_college_v2["Accept"] = df_college_v2.apply(lambda row:\
    slope_predict_accept*row["Apps"] + intercept_predict_accept if math.isnan(row["Accept"]) else row["Accept"], axis=1)

In [53]:
#Let's fill missing values now based on the linear model
df_college_v2["Apps"] = df_college_v2.apply(lambda row:\
    slope_predict_apps*row["Accept"] + intercept_predict_apps if math.isnan(row["Apps"]) else row["Apps"], axis=1)

In [54]:
df_college_v2

,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,PhD,S.F.Ratio,Grad.Rate
Abilene Christian University,1660.000000,1232.0,721,23,52,2885,537,70,18.1,60
Adelphi University,2186.000000,1924.0,512,16,29,2683,1227,29,12.2,56
Adrian College,1428.000000,1097.0,336,22,50,1036,99,53,12.9,54
Agnes Scott College,417.000000,349.0,137,60,89,510,63,92,7.7,59
Alaska Pacific University,213.322731,146.0,55,16,44,249,869,76,11.9,15
...,...,...,...,...,...,...,...,...,...,...
Worcester State College,2197.000000,1515.0,543,4,26,3089,2029,60,21.0,40
Xavier University,1959.000000,1805.0,695,24,47,2849,1107,73,13.3,83
Xavier University of Louisiana,2097.000000,1915.0,695,34,61,2793,166,67,14.4,49
Yale University,10705.000000,2453.0,1317,95,99,5217,83,96,5.8,99


In [55]:
df_college_v2.loc[missing_apps, ["Apps", "Accept"]] #we accepted more than the applications in some unis!
# all stable

,Apps,Accept
Alaska Pacific University,213.322731,146.0
Columbia College MO,231.160421,158.0
James Madison University,7852.313682,5285.0
Pacific Lutheran University,2065.469593,1392.0
University of the South,1432.231581,966.0
Virginia Tech,17416.288731,11719.0


In [56]:
df_college_v2.loc[missing_accepts, ["Apps", "Accept"]] #we accepted more than the applications in some unis!
# only firsty two have Accept > Apps but better than before!

,Apps,Accept
Chatham College,212.0,355.819002
Mount Mary College,235.0,369.521578
Muskingum College,1109.0,890.219466
Pennsylvania State Univ. Main Campus,19315.0,11736.702035
University of Delaware,14446.0,8835.926267
University of Georgia,11220.0,6913.991039
University of North Carolina at Asheville,1757.0,1276.274652
